In [ ]:
import sklearn.metrics
import numpy as np
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt

%matplotlib inline

# Стандартные метрики классификации

In [ ]:
iris_data = load_iris()

In [ ]:
pred = LogisticRegression().fit(iris_data.data, iris_data.target).predict(iris_data.data)

### Accuracy

- Accuracy - доля объектов, в которых предсказанный класс совпал с реальным

- Можно интерпретировать как вероятность правильно предсказать класс на случайно выбранном объекте

- Не репрезентативна при несбалансированных классах

In [ ]:
sklearn.metrics.accuracy_score(iris_data.target, pred)

### Precision

- Precision - количество правильно предсказанных объектов класса делённое на количество  объектов, которым этот класс был предсказан

- Можно интепретировать как вероятность того, что объект, отнесённый моделью к классу, окажется объектом этого класса.

- С точки зрения вероятности precision это $P(y = c \mid a(x) = c)$

In [ ]:
sklearn.metrics.precision_score(iris_data.target, pred)

In [ ]:
sklearn.metrics.precision_score(iris_data.target, pred, average=None)

### Recall

- Recall - количество правильно предсказанных объектов класса делённое на количество  объектов этого класса

- Можно интепретировать как вероятность того, что объект  класса будет отнесён моделью к этому классу

- С точки зрения вероятности recall это $P(a(x) = с \mid y = с)$

In [ ]:
sklearn.metrics.recall_score(iris_data.target, pred, average=None)

### F1 мера

- $f_1$ мера - способ скомбинировать recall и precision в одну метрику

- $f_1 = \frac{2~precsion~recall}{precision~+~recall}$

- Является средним гармоническим - $\frac{2}{f_1} = \frac{1}{precision} + \frac{1}{recall}$

- Имеет обобщение для учёта важности precision и recall $\frac{1 + \beta^2}{f_{\beta}} = \frac{1}{precision} + \frac{\beta^2}{recall} \implies f_{\beta} = \frac{(1+\beta^2)~precsion~recall}{\beta^2 precision~+~recall}$ 


In [ ]:
sklearn.metrics.f1_score(iris_data.target, pred, average=None)

In [ ]:
sklearn.metrics.fbeta_score(iris_data.target, pred, beta=1, average=None)

In [ ]:
sklearn.metrics.fbeta_score(iris_data.target, pred, beta=2, average=None)

Быстрый способ посмотреть сразу на все эти метрики - функция classification_report

(Прим.) support - количество объектов данного класса

In [ ]:
print(sklearn.metrics.classification_report(iris_data.target, pred, digits=4))

Довольно часто алгоритм выдаёт не просто метку класса, а вероятность / меру уверенности в классификации объекта, выставляя различные пороги для классов, мы получим не одну точку (precision, recall), а целый набор точек.

Тогда информативен совместный график precision-recall-threshold

Получить его можно при помощи функции precision_recall_curve

In [ ]:
probas = LogisticRegression().fit(iris_data.data, iris_data.target).predict_proba(iris_data.data)

In [ ]:
probas[:10, :]

In [ ]:
sklearn.metrics.precision_recall_curve(iris_data.target, probas)

Используем dataset с бинарным target-ом

In [ ]:
cancer_data = load_breast_cancer()

In [ ]:
probas = LogisticRegression().fit(cancer_data.data, cancer_data.target).predict_proba(cancer_data.data)

In [ ]:
probas[:10]

In [ ]:
sklearn.metrics.precision_recall_curve(cancer_data.target, probas)

In [ ]:
precision, recall, threshold = sklearn.metrics.precision_recall_curve(cancer_data.target, probas[:, 1])

In [ ]:
plt.xlabel('threshold')
plt.ylabel('recall')
plt.plot(threshold, recall[:-1])
plt.grid()
plt.show()

In [ ]:
plt.xlabel('threshold')
plt.ylabel('precision')
plt.plot(threshold, precision[:-1])
plt.grid()
plt.show()

In [ ]:
plt.xlabel('recall')
plt.ylabel('precision')
plt.plot(recall, precision)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(16, 5))
plt.subplot(1, 3, 1)
plt.xlabel('threshold')
plt.ylabel('recall')
plt.plot(threshold, recall[:-1])
plt.grid()

plt.subplot(1, 3, 2)
plt.xlabel('threshold')
plt.ylabel('precision')
plt.plot(threshold, precision[:-1])
plt.grid()

plt.subplot(1, 3, 3)
plt.xlabel('recall')
plt.ylabel('precision')
plt.plot(recall, precision)
plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(11, 5))
plt.subplot(1, 2, 1)
plt.xlabel('threshold')
plt.ylabel('value')
plt.plot(threshold, recall[:-1])
plt.plot(threshold, precision[:-1])
plt.legend(['recall', 'precision'])
plt.grid()

plt.subplot(1, 2, 2)
plt.xlabel('recall')
plt.ylabel('precision')
plt.plot(recall, precision)
plt.grid()

plt.show()

- Ещё один общепринятый способ скомбинировать precision и recall - подсчитать площадь под precision-recall прямой

- AUC - Area Under Curve, поэтому данную метрику также называют PR AUC

- В sklearn этот метод называется average_precision_score

In [ ]:
sklearn.metrics.average_precision_score(cancer_data.target, probas[:, 1])

### ROC-AUC

Помимо precision-recall кривой так же часто используют roc кривую

Чтобы определить roc кривую нужно ввести дополнительные обозначения:

- TP - True Positive - #$\{a(x) = 1, y = 1\}$

- FP - False Positive - #$\{a(x) = 1, y = 0\}$

- TN - True Negative - #$\{a(x) = 0, y = 0\}$

- FN - False Negative - #$\{a(x) = 0, y = 1\}$

Тогда:

- TPR - True Positive Rate - $\frac{TP}{TP + FN}$

- FPR - False Positive Rate - $\frac{FP}{FP + TN}$

ROC кривая это набор точек $(FPR, TPR)$, полученный варирование порога


**Что такое precsion, recall и accuracy в данных обозначениях?**

In [ ]:
sklearn.metrics.confusion_matrix(cancer_data.target, probas[:, 1] > 0.5)

In [ ]:
sklearn.metrics.confusion_matrix(iris_data.target, pred)

In [ ]:
(
    (tp, fp),
    (fn, tn)
) = sklearn.metrics.confusion_matrix(cancer_data.target, probas[:, 1] > 0.5)

In [ ]:
tp / (tp + fn), fp / (fp + tn)

In [ ]:
fpr, tpr, threshold = sklearn.metrics.roc_curve(cancer_data.target, probas[:, 1])

In [ ]:
tpr[fpr <= 0.03867403315].max()

In [ ]:
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.plot(fpr, tpr)
plt.grid()
plt.show()

- ROC AUC - площадь под данной кривой

- В sklearn считается функцией roc_auc_score

In [ ]:
sklearn.metrics.roc_auc_score(cancer_data.target, probas[:, 1])

- **Чему равен ROC AUC случайного предсказания? То есть proba - равномерно распредлённая на отрезке случайная величина**
- **Чему равен ROC AUC константного предсказания?**

In [ ]:
sklearn.metrics.roc_auc_score(cancer_data.target, np.zeros_like(cancer_data.target))

In [ ]:
sklearn.metrics.roc_auc_score(cancer_data.target, np.random.uniform(size=len(cancer_data.target)))

### log loss

- Минус логарифм правдоподобия выборки $log loss = -\frac1n \sum\limits_{i=1}^n y_i \log a(x_i) + (1 - y_i) \log (1 - a(x_i))$

In [ ]:
sklearn.metrics.log_loss(cancer_data.target, probas[:, 1])

# Стандартные метрики регрессии

- Общая концепция - $\frac1n \sum\limits_{i=1}^n L(a(x_i), y_i)$

-  MAE - Mean Absolute Error - $\frac1n \sum\limits_{i=1}^n |a(x_i) - y_i|$

- MSE - Mean Squared Error - $\frac1n \sum\limits_{i=1}^n (a(x_i) - y_i)^2$

- MAPE - Mean Absolute Percentage Error - $\frac1n \sum\limits_{i=1}^n \frac{|a(x_i) - y_i|}{y_i}$

- sMAPE - symmetric Mean Absolute Percentage Error - $\frac2n \sum\limits_{i=1}^n \frac{|a(x_i) - y_i|}{a(x_i) + y_i}$

In [ ]:
data = load_diabetes()

In [ ]:
cross_val_score(RandomForestRegressor(), data.data, data.target, cv=3, scoring='neg_mean_absolute_error')

In [ ]:
cross_val_score(RandomForestRegressor(), data.data, data.target, cv=3, scoring='neg_mean_squared_error')

In [ ]:
cross_val_score(RandomForestRegressor(), data.data, data.target, cv=3, scoring='neg_mean_absolute_percentage_error')

In [ ]:
def absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / y_true)

In [ ]:
def symmetric_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / (y_true + y_pred))

In [ ]:
cross_val_score(
    RandomForestRegressor(), 
    data.data, data.target, 
    cv=3, 
    scoring=sklearn.metrics.make_scorer(absolute_percentage_error, greater_is_better=False)
)

In [ ]:
cross_val_score(
    RandomForestRegressor(random_state=42), 
    data.data, data.target, 
    cv=3,
    scoring=sklearn.metrics.make_scorer(symmetric_absolute_percentage_error, greater_is_better=False)
)

In [ ]:
def smape_scorer(estimator, X, y):
    pred = estimator.predict(X)
    return - symmetric_absolute_percentage_error(y, pred)

In [ ]:
cross_val_score(
    RandomForestRegressor(random_state=42), 
    data.data, data.target, 
    cv=3, 
    scoring=smape_scorer
)

In [ ]:
cross_validate(
    RandomForestRegressor(random_state=42), 
    data.data, data.target, 
    cv=3, 
    scoring={
        'smape': smape_scorer,
        'mse': sklearn.metrics.make_scorer(sklearn.metrics.mean_squared_error, greater_is_better=False)
    },
    return_train_score=True
)

### Несколько свойств оптимизации метрик регрессии

- $\sum\limits_{i=1}^n \left(a(x_i) - y_i\right)^2 \to \min \implies a(x_i) \approx E(y \mid x = x_i)$

- $\sum\limits_{i=1}^n \left|~a(x_i) - y_i~\right| \to \min \implies a(x_i) \approx median(y \mid x = x_i)$

- $\sum\limits_{i=1}^n \alpha~(a(x_i) - y_i)~I\{a(x_i) \geq y_i\} + (1-\alpha)~(y_i - a(x_i))~I\{a(x_i) < y_i\} \to \min \implies a(x_i) \approx Z_{1 - \alpha}(y \mid x = x_i)$

**Докажите утверждения в случаях, если все $x_i$ одинаковые**

### Как оптимизировать MAPE через оптимизацию MAE?

**Подберите такую функцию $f$, чтобы**

$\sum\limits_{i=1}^n \left|~f(a(x_i)) - f(y_i)~\right| \approx \sum\limits_{i=1}^n \frac{|a(x_i) - y_i|}{y_i}$

Возможно, стоит разложить левую часть в ряд Тейлора